In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")


In [ ]:
train = pd.read_csv("/kaggle/input/playground-series-s6e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")
sample_sub = pd.read_csv("/kaggle/input/playground-series-s6e2/sample_submission.csv")

print(train.shape, test.shape)
train.head()


In [ ]:
train.info()
train.describe()
train["Heart Disease"].value_counts(normalize=True)


In [ ]:
TARGET = "Heart Disease"
ID_COL = "id"

X = train.drop(columns=[TARGET, ID_COL])
y = train[TARGET]
X_test = test.drop(columns=[ID_COL])


In [ ]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

# Label Encoding for categorical
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    X_test[col] = le.transform(X_test[col])


In [ ]:
imputer = SimpleImputer(strategy="median")

X[num_cols] = imputer.fit_transform(X[num_cols])
X_test[num_cols] = imputer.transform(X_test[num_cols])


In [ ]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    
    print(f"Training Fold {fold+1}")
    
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    model = lgb.LGBMClassifier(
        n_estimators=200,
        learning_rate=0.01,
        max_depth=-1,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(200, verbose=False)]
    )
    
    oof_preds[valid_idx] = model.predict_proba(X_valid)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / folds.n_splits
    
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f"Fold {fold+1} AUC: {fold_auc:.5f}")


In [ ]:
overall_auc = roc_auc_score(y, oof_preds)
print("Overall CV AUC:", overall_auc)


In [ ]:
submission = pd.DataFrame({
    "id": test[ID_COL],
    "Heart Disease": test_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.01,
    depth=6,
    eval_metric="AUC",
    verbose=200
)

model.fit(X_train, y_train, eval_set=(X_valid, y_valid))


In [ ]:
import os
print(os.listdir())
